<a href="https://colab.research.google.com/github/bxck75/A1_Colabs/blob/master/New_piss_ant_pix2pixV0_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Get images and make the dataset
    --resize
    --blank center clone
    --edged clone
    --combine blank set
    --combine edged set
    --split both sets
    --copy the combined set to there home 

In [0]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

# Helpers Loading
from pathlib import Path
lib_file=Path('/content/lib/Helpers.py')
if not lib_file.is_file():
    import os
    os.system('mkdir -p /content/lib')
    os.chdir('/content/lib')
    os.system('wget https://raw.githubusercontent.com/bxck75/A1_Colabs/master/Helpers.py')
    os.system('wget https://raw.githubusercontent.com/bxck75/A1_Colabs/master/myrepcol.py')
    os.chdir('/content/')

# #resource monitor
# resmon_file = Path('/content/lib/resource_monitor.py')
# if not  resmon_file.is_file():
#     os.system('wget https://github.com/googlecolab/colabtools/blob/master/google/colab/_serverextension/_resource_monitor.py -O /content/lib/resource_monitor.py')

# # import lib.resource_monitor
    
# patch for imshow
patch_file = Path('/content/lib/colab_patches.py')
if not patch_file.is_file():
    os.system('wget https://raw.githubusercontent.com/googlecolab/colabtools/master/google/colab/patches/__init__.py -O /content/lib/colab_patches.py')

import lib.colab_patches
from lib.Helpers import Helpers
from lib.myrepcol import reps

# regular imports
import ipywidgets as widgets
from ipywidgets import HBox, VBox
from fastai.vision import *
from fastai.vision.gan import *
import seaborn as sns
import numpy as np
import pandas as pd
import fastai, hashlib, inspect
from IPython.display import clear_output
from IPython import display

# Init helpers
H=Helpers()

# delete standaard data
H.Me(['cml','rm -r /content/sample_data'])

# login google session
from google.colab import drive
drive.mount('/content/drive')

# define functions
def get_gdrive_dataset(pack, DS_root='datasets',GD_root='datasets'):
    import google
    from google.colab import drive
    drive.mount('/content/drive', force_remount=True)
    H.GD_ROOT=GD_root+'/'
    H.DS_ROOT=DS_root+'/'
    os.chdir(H.gdrive_root+H.GD_ROOT)
    H.Me(['mkd',[DS_root,'models'],H.pix_root])
    H.Me(['cml','cp -r '+pack+' '+H.pix_root+DS_root])
    os.chdir(H.pix_root+DS_root)
    H.Me(['cml','unzip -q '+pack])
    H.Me(['cml','rm -r '+pack])
    os.chdir(H.pix_root)

def MethHelp(libs):
    os_help=H.Me(['vdir',libs])
    #make a list containing libs values of os_help
    listOfLibs = [x[0] for x in os_help]
    #make a list containing libs method values of os_help
    listOfMethods= [x[1] for x in os_help]
    # Create a zipped list of tuples from above lists
    zippedList =  list(zip(listOfLibs, listOfMethods))
    zippedList
    # request help on method from list
    return zippedList

def loadTboard():
    import datetime, os
    try:
        %load_ext tensorboard
    except:
        %reload_ext tensorboard

def html(content):
    """Publishes given html content into the output."""
    display.display(display.HTML(content))


def css(content=None, url=None):
    """Publishes css content."""
    if url is not None:
        html('<link rel=stylesheet type=text/css href=%r></link>' % url)
    else:
        html('<style>' + content + '</style>')


def javascript(content=None, url=None, script_id=None):
    """Publishes javascript content into the output."""
    if (content is None) == (url is None):
        raise ValueError('exactly one of content and url should be none')
    if url is not None:
        # Note: display.javascript will try to download script from python
        # which is very rarely useful.
        html('<script src=%r></script>' % url)
        return
    if not script_id and 'sourceURL=' not in content:
        script_id = 'js_' + hashlib.md5(content.encode('utf8')).hexdigest()[:10]

    if script_id:
        content += '\n//# sourceURL=%s' % script_id
    display.display(display.Javascript(content))

# set repository to pull
H.reps_custom=['bxck75/piss-ant-pix2pix']# ,'affinelayer/pix2pix-tensorflow','phillipi/pix2pix','shelhamer/clockwork-fcn'
H.reps_all=reps # aprox 80 in lib reps

H.Me(['inst_reps',H.reps_custom,'/content/installed_reps',True,True])

H.pix_root='/content/installed_reps/piss-ant-pix2pix/'
H.gdrive_root='/content/drive/My Drive/'
H.local_datasets_path=H.pix_root+'datasets/'
H.gdrive_datasets_path=H.gdrive_root+'datasets/'
H.local_raw_imgs_path=H.pix_root+'raw_img_data/'
H.gdrive_raw_imgs_path=H.gdrive_root+'raw_image_packs/'
H.gdrive_models_path=H.gdrive_root+'models/'

# resource lists
H.raw_image_packs=H.Me(['cml','ls "'    + H.gdrive_raw_imgs_path+'"'])
H.processed_datasets=H.Me(['cml','ls "' + H.gdrive_datasets_path+'"'])
H.trained_models=H.Me(['cml','ls "'     + H.gdrive_models_path+'"'])

In [0]:
# set img input method
H.img_input_method="dataset"        #@param ['raw','gallery-dl','dataset']
# set new or re train
H.pretrained_or_new_model='new' #@param ['new','pre']
# name of the dataset
H.dataset_name='catsanddogs'         #@param {type : "string"}
# size
H.resize_value=32              #@param {type : ""}
# epochs
H.epochs_value=1              #@param {type : ""}

## Process a folder of images

In [0]:

if (H.img_input_method =='raw'):
    print('This inits function ProcessRawImgFolder')
    ProcessRawImgFolder()
    
if(H.img_input_method =='dataset'):
    FetchSet()
    
def ProcessRawImgFolder():   
    # size
#     H.resize_value=256
    # get drive files
    get_gdrive_dataset(H.dataset_name+'.zip','raw_img_data','raw_image_packs')

    # mangafaces fix
    if H.dataset_name == 'mangafaces':
        # images are in folder data and need to be in mangafaces
        H.Me(['cml','mkdir -p /content/installed_reps/piss-ant-pix2pix/raw_img_data/'+H.dataset_name])
        H.Me(['cml','cp -r /content/installed_reps/piss-ant-pix2pix/raw_img_data/data/* /content/installed_reps/piss-ant-pix2pix/raw_img_data/'+H.dataset_name])
        H.Me(['cml','rm -r /content/installed_reps/piss-ant-pix2pix/raw_img_data/data'])

      # set path to images to process
    IntoPath=H.local_raw_imgs_path + H.dataset_name
    H.img_path=Path(IntoPath)

    # goto pix root
    os.chdir(H.pix_root)
    print('Image folder set to '+str(IntoPath))

    # glob the folder for images
    H.prepro_img_list=H.Me(['globx',IntoPath,'*.png'])
    print(len(H.prepro_img_list),end=' ')
    print(' Images Found!')

    # make the processing folders 
    H.Me(['mkd',['_blank', '_edged', '_resized', '_combined_blank', '_combined_edged'], H.pix_root+'p2p_process_tmp'])
    # make the dataset folders
    H.Me(['mkd',['_resize_edged_combined', '_resize_blank_combined', '_resized', '_edged', '_blank'], H.local_datasets_path+H.dataset_name])
    # process
    H.Me(['cml','python tools/process.py --input_dir '+IntoPath+' --operation resize --size '+str(H.resize_value)+' --output_dir '+H.pix_root+'p2p_process_tmp/_resized/'])
    H.Me(['cml','python tools/process.py --input_dir '+H.pix_root+'p2p_process_tmp/_resized --operation blank --output_dir '+H.pix_root+'p2p_process_tmp/_blank'])
    H.Me(['cml','python tools/edge.py    --input_dir '+H.pix_root+'p2p_process_tmp/_resized --output_dir '+H.pix_root+'p2p_process_tmp/_edged'])
    # combine A/B
    H.Me(['cml','python tools/process.py --input_dir '+H.pix_root+'p2p_process_tmp/_resized --b_dir '+H.pix_root+'p2p_process_tmp/_edged --operation combine --output_dir '+H.pix_root+'p2p_process_tmp/_combined_edged'])
    H.Me(['cml','python tools/process.py --input_dir '+H.pix_root+'p2p_process_tmp/_resized --b_dir '+H.pix_root+'p2p_process_tmp/_blank --operation combine --output_dir '+H.pix_root+'p2p_process_tmp/_combined_blank'])
    # split
    H.Me(['cml','python tools/split.py --dir '+H.pix_root+'p2p_process_tmp/_combined_edged'])
    H.Me(['cml','python tools/split.py --dir '+H.pix_root+'p2p_process_tmp/_combined_blank'])
    H.Me(['cml','python tools/split.py --dir '+H.pix_root+'p2p_process_tmp/_resized'])
    H.Me(['cml','python tools/split.py --dir '+H.pix_root+'p2p_process_tmp/_blank'])
    H.Me(['cml','python tools/split.py --dir '+H.pix_root+'p2p_process_tmp/_edged'])
    # copy over to permanent datasets folder
    H.Me(['cml','cp -rf '+H.pix_root+'p2p_process_tmp/_resized/* '+H.local_datasets_path+H.dataset_name+'/_resized/'])
    H.Me(['cml','cp -rf '+H.pix_root+'p2p_process_tmp/_edged/* '+H.local_datasets_path+H.dataset_name+'/_edged/'])
    H.Me(['cml','cp -rf '+H.pix_root+'p2p_process_tmp/_blank/* '+H.local_datasets_path+H.dataset_name+'/_blank/'])
    H.Me(['cml','cp -rf '+H.pix_root+'p2p_process_tmp/_combined_edged/* '+H.local_datasets_path+H.dataset_name+'/_resize_edged_combined/'])
    H.Me(['cml','cp -rf '+H.pix_root+'p2p_process_tmp/_combined_blank/* '+H.local_datasets_path+H.dataset_name+'/_resize_blank_combined/'])
    # set as active set
    H.ActiveSetPath = H.local_datasets_path+H.dataset_name+'/_resize_edged_combined/'
    # zip set to gdrive
    os.chdir(H.local_datasets_path+H.dataset_name)
    H.Me(['cml','zip -r "'+H.gdrive_datasets_path+H.dataset_name+'_set.zip" ./*'])
    os.chdir(H.pix_root)
    RunWithNewBrain()


## Load a pre processed set from drive

In [0]:

# H.dataset_name='cityscapes_set'
# get_gdrive_dataset(H.dataset_name+'.zip','datasets','/content/drive/My Drive/datasets/')
# result=H.Me(['vdir'])
# H.local_dataset_root + H.dataset_name
# print(H.method_args)
# print(result)


# H.dataset_name='elvis_edged_set'
# get_gdrive_dataset(H.dataset_name+'.zip','datasets','/content/drive/My Drive/datasets/')
# result=H.Me(['vdir'])
# H.local_dataset_root + H.dataset_name
# print(H.method_args)
# print(result)

# H.dataset_name='elvis_edged_set'
# get_gdrive_dataset(H.dataset_name+'.zip','datasets/elvis','/content/drive/My Drive/datasets/')
# result=H.Me(['vdir'])
# H.local_dataset_root + H.dataset_name
# print(H.method_args)
# print(result)


# # dataset
# H.dataset_name='pail'
# !mkdir -p {H.pix_root}dataset_test
# get_gdrive_dataset('/content/drive/My Drive/Colab Notebooks/datasets/'+H.dataset_name+'_datasets.zip','dataset_test')
# H.dataset_path=Path(H.ActiveSetPath)
# H.Me(['cml','cp -rf '+H.pix_root+'/dataset_test/content/installed_reps/piss-ant-pix2pix/dataset_test/* '+ H.ActiveSetPath+'/'])
# # H.Me(['cml','rm -r /content/installed_reps/piss-ant-pix2pix/dataset_test/content '])
# Zip the model to gdrive
 print(H.gdrive_models_path+H.dataset_name+'_model_training.zip')

## Load pretrained model and dataset files



In [0]:
def FetchSet():
    # get the preprocessed dataset
#     H.dataset_name='insect'
    get_gdrive_dataset(H.dataset_name+'_set.zip','datasets','datasets')
    H.ActiveSetPath=H.local_datasets_path + H.dataset_name+'/_resize_edged_combined/train'
    H.dataset_path=Path(H.ActiveSetPath)
    # count the images in combine folder
    H.check_image_folder=H.Me(['globx',H.ActiveSetPath,'*.png'])
    count=len(H.check_image_folder)
    if count > 0:
        print(count,end=' ')
        print(' Images Found!')
        if H.pretrained_or_new_model == 'pre':
            # download pretrained brain
            H.BrainZipFile=H.dataset_name+'_model_training.zip'
            f=PATH(H.gdrive_models_path+H.dataset_name+'_model_training.zip')
            if not f.is_file():
                print('The brain ('+H.BrainZipFile+') is not found!')
                exit()
            else:
                print("Found the brain ('+H.BrainZipFile+') fetching it now from gdrive!")
                FetchModel()
        else:
            RunWithNewBrain()

def FetchModel():
    # getting the file from gdrive
    get_gdrive_dataset(H.dataset_name+'_model_training.zip','models','models')
    H.ActiveModelPath=H.pix_root +'models/'+H.dataset_name+'/model'
    # see if the index.html is working
    os.chdir(H.pix_root +'models/'+H.dataset_name+'/model')
    display.display(display.HTML('index.html'))  
    os.chdir(H.pix_root)
    
def RunWithNewBrain():
    # Load Tensorboard
    from tensorboard import notebook
#     notebook.list()
    %reload_ext tensorboard
    import tensorflow as tf
    %tensorboard --logdir {output_folder}
    
    training_epochs=H.epochs_value
    mode='train'
    direction='BtoA'
    lr=5e-05
    print('Learn rate set on '+ str('{:f}'.format(lr)))
    # lr = 9.12E-04
    !python pix2pix.py  --mode {mode} --batch_size 2 --summary_freq 10 --progress_freq 10 --save_freq 50 --display_freq 10 --output_dir {output_folder} --max_epochs {str(training_epochs)} --input_dir {input_folder} --which_direction {direction}
    # H.Me(['cml','python pix2pix.py  --mode '+mode+' --batch_size 4 --summary_freq 1 --progress_freq 10 --save_freq 100 --display_freq 10 --output_dir '+output_folder+' --max_epochs '+str(training_epochs)+' --input_dir '+input_folder+' --which_direction '+direction])
    # zip To gdrive
    os.chdir('cd /content/installed_reps/piss-ant-pix2pix/models')
    H.Me(['cml','zip -r "'+H.gdrive_root+'/models/'+H.dataset_name+'_'+str(training_epochs)+'_epoch_training.zip" ./*'])
    print('Trained Brain was saved under  ' + H.gdrive_root+'/models/'+H.dataset_name+'_'+str(training_epochs)+'_epoch_training.zip')
      
def RunWithPretrainedBrain():
    # Load Tensorboard
    from tensorboard import notebook
#     notebook.list()
    %reload_ext tensorboard
    import tensorflow as tf
    %tensorboard --logdir {output_folder}
    # not doing real epochs but for loop on 2 epochs then zipping to gdrive because of banning and disconnecting
    for i in range(H.epochs_value):
        training_epochs=2
        mode='train'
        direction='BtoA'
        lr=5e-05
        lr2 = 9.12E-04
        print( 'Learn rate set on ' + "%.6f" % float(lr2))
        # get the checkpoint dir
        start_with_checkpoint = '--checkpoint '+output_folder # '+with_checkpoint+' 

        !python pix2pix.py {start_with_checkpoint} --mode {mode} --batch_size 2 --summary_freq 10 --progress_freq 10 --save_freq 50 --display_freq 10 --output_dir {output_folder} --max_epochs {str(training_epochs)} --input_dir {input_folder} --which_direction {direction}
        # H.Me(['cml','python pix2pix.py '+start_with_checkpoint+' --mode '+mode+' --batch_size 16 --summary_freq 50 --progress_freq 100 --save_freq 100 --display_freq 5 --output_dir '+output_folder+' --max_epochs '+str(training_epochs)+' --input_dir '+input_folder+' --which_direction '+direction])
        # zipping to gdrive
        os.chdir('cd /content/installed_reps/piss-ant-pix2pix/models')
        H.Me(['cml','zip -r "'+H.gdrive_root+'/models/'+H.dataset_name+'_'+str(training_epochs)+'_epoch_training.zip" ./*'])


In [0]:
# TRAIN!!!!!
output_folder=H.pix_root+'models/'+H.dataset_name+'/model'
input_folder = H.ActiveSetPath

In [0]:
import pandas as pd
pd.options.display.float_format = '{:.2f}'.format
lr=5e-05



In [0]:

# print(H.ActiveSetPath)
# H.Me(['vdir'])
from tensorboard import notebook
notebook.list()
%reload_ext tensorboard
import tensorflow as tf
%tensorboard --logdir {output_folder}

# Training

### help
>['aspect_ratio = 1.0',
 'batch_size = 16',
 'beta1 = 0.5',
 'checkpoint = /content/installed_reps/piss-ant-pix2pix/garbagepailkids_resize/training',
 'display_freq = 1',
 'flip = True',
 'gan_weight = 1.0',
 'input_dir = /content/installed_reps/piss-ant-pix2pix/garbagepailkids_resize/_combined/train',
 'l1_weight = 0.0',
 'lab_colorization = False',
 'lr = 5e-05',
 'max_epochs = 10',
 'max_steps = None',
 'mode = train',
 'ndf = 64',
 'ngf = 64',
 'output_dir = /content/installed_reps/piss-ant-pix2pix/garbagepailkids_resize/training',
 'patch_gan = 1',
 'progress_freq = 1',
 'save_freq = 100',
 'scale_size = 266',
 'seed = 0',
 'skip_connection = 1',
 'summary_freq = 1',
 'trace_freq = 0',
 'wgan = 1',
 'which_direction = BtoA',
 '/content/installed_reps/piss-ant-pix2pix/garbagepailkids_resize/_combined/train']

In [0]:
training_epochs=5
mode='train'
direction='BtoA'
lr=5e-05
# lr = 9.12E-04
!python pix2pix.py  --mode {mode} --batch_size 4 --summary_freq 10 --progress_freq 10 --save_freq 10 --display_freq 10 --output_dir {output_folder} --max_epochs {str(training_epochs)} --input_dir {input_folder} --which_direction {direction}

# H.Me(['cml','python pix2pix.py  --mode '+mode+' --batch_size 4 --summary_freq 1 --progress_freq 10 --save_freq 100 --display_freq 10 --output_dir '+output_folder+' --max_epochs '+str(training_epochs)+' --input_dir '+input_folder+' --which_direction '+direction])


In [0]:


H.Me(['cml','zip -r "'+H.gdrive_root+'/models/'+H.dataset_name+'_'+str(training_epochs)+'_epoch_training.zip" '+output_folder+'/*'])
%tensorboard --logdir={output_folder}


In [0]:

training_epochs=10
mode='train'
direction='BtoA'
lr=5e-05
# lr = 9.12E-04
# get the checkpoint dir


start_with_checkpoint = '--checkpoint '+output_folder # '+with_checkpoint+' 

!python pix2pix.py {start_with_checkpoint} --mode {mode} --batch_size 4 --summary_freq 10 --progress_freq 10 --save_freq 10 --display_freq 10 --output_dir {output_folder} --max_epochs {str(training_epochs)} --input_dir {input_folder} --which_direction {direction}

# H.Me(['cml','python pix2pix.py '+start_with_checkpoint+' --mode '+mode+' --batch_size 16 --summary_freq 50 --progress_freq 100 --save_freq 100 --display_freq 5 --output_dir '+output_folder+' --max_epochs '+str(training_epochs)+' --input_dir '+input_folder+' --which_direction '+direction])



In [0]:
# Zip the model to gdrive
H.Me(['cml','zip -r "'+H.gdrive_root+'/models/'+H.dataset_name+'_'+str(training_epochs)+'_epoch_training.zip" '+output_folder+'/*'])
# H.Me(['cml','unzip "'+H.gdrive_root+'/models/'+H.dataset_name+'_'+str(training_epochs)+'_epoch_training.zip" -d '+output_folder+'_pretrained'])

In [0]:

%cd /content/installed_reps/piss-ant-pix2pix/models
H.Me(['cml','zip -r "'+H.gdrive_root+'/models/'+H.dataset_name+'_'+str(training_epochs)+'_epoch_training.zip" ./*'])

# checkpoints
# /content/drive/My\\ Drive/Colab\\ Notebooks/pretrained_models/apes_1_epoch_training.zip
# stored_models=H.Me(['globx',output_folder,'*outputs.png'])
# print(stored_models)
# !ls -l /content/installed_resp/piss-ant-pix2pix/'+H.pix_root+'_resize/training_train

In [0]:
# !wget https://live.staticflickr.com/2936/33549790582_48a514baed_b.jpg

In [0]:
# /content/installed_resp/piss-ant-pix2pix/33549790582_48a514baed_b.jpg
# !python pix2pix.py --mode test --output_dir /content/installed_resp/piss-ant-pix2pix/'+H.pix_root+'_resize/test --input_dir /content/installed_resp/piss-ant-pix2pix --seed 0 --checkpoint /content/installed_resp/piss-ant-pix2pix/'+H.pix_root+'_resize/training_train


In [0]:
# save the complete model
# !python /content/installed_resp/piss-ant-pix2pix/server/tools/export-checkpoint.py --checkpoint /content/installed_resp/piss-ant-pix2pix/'+H.pix_root+'_resize/_combined/train --output_file /content/installed_resp/piss-ant-pix2pix/'+H.pix_root+'_resize/training_train/'+H.pix_root+'_BtoA_model_save.bin

In [0]:

# # store zip to drive
# !zip /content/installed_resp/piss-ant-pix2pix/'+H.pix_root+'/* /content/installed_resp/piss-ant-pix2pix/'+H.pix_root+'_original_images.zip
# !zip /content/installed_resp/piss-ant-pix2pix/'+H.pix_root+'_resize/_combined/* /content/installed_resp/piss-ant-pix2pix/'+H.pix_root+'_dataset.zip
# !zip /content/installed_resp/piss-ant-pix2pix/'+H.pix_root+'_resize/training_train/'+H.pix_root+'_BtoA_model_save.bin /content/installed_resp/piss-ant-pix2pix/'+H.pix_root+'_model_bin.zip
# !zip /content/installed_resp/piss-ant-pix2pix/'+H.pix_root+'_resize/training_train/* /content/installed_resp/piss-ant-pix2pix/'+H.pix_root+'.zip

# %cd /content/drive/My\ Drive/Colab\ Notebooks/Best_Colabs
# !cp /content/installed_resp/piss-ant-pix2pix/'+H.pix_root+'_original_images.zip '+H.pix_root+'_original_images.zip
# !cp /content/installed_resp/piss-ant-pix2pix/'+H.pix_root+'_dataset.zip ./'+H.pix_root+'_dataset.zip
# !cp /content/installed_resp/piss-ant-pix2pix/'+H.pix_root+'_model_bin.zip ./'+H.pix_root+'_model_bin.zip
# !cp /content/installed_resp/piss-ant-pix2pix/'+H.pix_root+'.zip ./'+H.pix_root+'.zip

## pretrained
